<a href="https://colab.research.google.com/github/myers1091/UmpqQuant_GroupLearningSeminar/blob/main/hnorem/session04/Ch_12_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras

%load_ext tensorboard

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

## exercise 3

In [ ]:
tf.range(10)

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [ ]:
tf.constant(np.arange(10))

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])>

In [ ]:
## Exercise 12

In [ ]:
class normalization_layer(keras.layers.Layer):
  def __init__(self, smoothing = 0.001, **kwargs):
    self.smoothing = smoothing
    super().__init__(**kwargs)


  def build(self, input_shape):
    self.alpha = self.add_weight(name='alpha', 
                                 shape=input_shape[-1:],
                                 dtype=tf.float32,
                                 initializer="ones")
    self.beta = self.add_weight(name='beta', 
                                 shape=input_shape[-1:],
                                 dtype=tf.float32,
                                 initializer="zeros")
    super().build(input_shape)

  def call(self, inputs):
    mean, variance = tf.nn.moments(inputs, axes=-1, keepdims=True)
    #print(mean, variance)
    return self.alpha * (inputs - mean) / (tf.sqrt(variance + self.smoothing)) + self.beta
    #return self.alpha * ((input - mean)/tf.sqrt(variance + self.smoothing)) + self.beta <--- THIS ONE DIDNT WORK?

  def compute_output_shape(self, batch_input_shape):
    return batch_input_shape

  def get_config(self):
    base_config = super().get_config()
    return {**base_config, "smoothing": self.smoothing}

In [ ]:
tf.sqrt(tf.constant([4.]))


<tf.Tensor: shape=(1,), dtype=float32, numpy=array([2.], dtype=float32)>

In [ ]:
(tf.sqrt(tf.constant([4.])))


<tf.Tensor: shape=(1,), dtype=float32, numpy=array([2.], dtype=float32)>

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

In [ ]:
X = X_train.astype(np.float32)

custom_layer_norm = normalization_layer()
keras_layer_norm = keras.layers.LayerNormalization()

tf.reduce_mean(keras.losses.mean_absolute_error(
    keras_layer_norm(X), custom_layer_norm(X)))

<tf.Tensor: shape=(), dtype=float32, numpy=4.1984418e-08>